# 第四、五模块：重排与回复生成

本 Notebook 实现酒店评论 RAG 系统的重排与回复生成流程及完整系统集成：

### 模块四：重排
1. **Rerank 相关性打分**: 使用 Qwen3-Rerank 模型计算语义相关性
2. **内容质量特征**: 评论质量分、评论长度、回复数、点赞数
3. **时效性衰减**: 基于发布日期的指数衰减（衰减率受意图检测中的时效性需求影响）
4. **多因子融合**: 线性加权综合各维度得分

### 模块五：回复生成
1. **直接回答**: 对于不需要检索的问题，直接生成回复
2. **检索增强**: 基于检索上下文（评论+摘要）生成回复

### 完整 RAG 系统
- 集成模块一至五的完整工作流
- 支持意图扩展、五路召回、重排、回复生成模块的开关配置
- 支持系统主要超参数配置（模型选择、检索参数、重排参数）

**前置条件**：
- 已完成模块一至三的全部实现
- 环境变量：`DASHSCOPE_API_KEY`, `DASHVECTOR_API_KEY`, `DASHVECTOR_HOTEL_ENDPOINT`

## 环境配置

In [1]:
import os
import time
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

# API 客户端
from dashscope import Generation, TextReRank
import dashvector
import chromadb

from lib import (
    LLMClient, EmbeddingClient, InvertedIndex,                                                                             # 导入模块一的类
    IntentRecognizer, IntentDetector, IntentExpander, HyDEGenerator, HybridRetriever, EXACT_ROOM_TYPES, FUZZY_ROOM_TYPES,  # 导入模块二、三的类和常量
    print_rag_result
)

In [2]:
# 检查环境变量
required_env = {
    "DASHSCOPE_API_KEY": os.getenv("DASHSCOPE_API_KEY"),
    "DASHVECTOR_API_KEY": os.getenv("DASHVECTOR_API_KEY"),
    "DASHVECTOR_HOTEL_ENDPOINT": os.getenv("DASHVECTOR_HOTEL_ENDPOINT"),
}

missing = [k for k, v in required_env.items() if not v]
if missing:
    raise EnvironmentError(f"缺少环境变量: {', '.join(missing)}")

print("环境变量检测成功:")
for key, value in required_env.items():
    print(f"- {key}: {value[:10]}...{value[-4:]}")

环境变量检测成功:
- DASHSCOPE_API_KEY: sk-5540c66...f37a
- DASHVECTOR_API_KEY: sk-W6G93uB...7EDE
- DASHVECTOR_HOTEL_ENDPOINT: vrs-cn-e4k....com


In [3]:
# 项目路径配置
PROJECT_ROOT = Path.cwd()
DATA_DIR = PROJECT_ROOT / "data"

# 模型配置
DETECTION_MODEL = "qwen-plus"
EXPANSION_HYDE_MODEL = "qwen-flash"
GENERATION_MODEL = "deepseek-v3.2"

# 当前时间
TODAY = datetime(2025, 4, 17)

print(f"项目根目录: {PROJECT_ROOT}")
print(f"意图检测模型: {DETECTION_MODEL}")
print(f"意图扩展与HyDE生成模型: {EXPANSION_HYDE_MODEL}")
print(f"回复生成模型: {GENERATION_MODEL}")
print(f"时间基准: {TODAY}")

项目根目录: C:\Users\22418\Desktop\Scorpio\复旦\课程\大三下\酒店评论
意图检测模型: qwen-plus
意图扩展与HyDE生成模型: qwen-flash
回复生成模型: deepseek-v3.2
时间基准: 2025-04-17 00:00:00


## 模块四：重排

### 4.1 Rerank 相关性打分

In [4]:
class Reranker:
    """Reranker：使用 Qwen3-Rerank 模型计算相关性得分"""
    
    def __init__(self, api_key: str, model: str = "qwen3-rerank"):
        self.api_key = api_key
        self.model = model
    
    def rerank(self, query: str, documents: list[str], topk: int = None) -> dict:
        """
        对文档进行重排序
        
        参数:
            query: 查询文本
            documents: 待排序的文档列表
            topk: 返回前 K 个结果（默认返回全部）
        
        返回:
            {index: relevance_score}
        """
        if topk is None:
            topk = len(documents)
        
        response = TextReRank.call(
            api_key=self.api_key,
            model=self.model,
            query=query,
            documents=documents,
            top_n=topk,
            return_documents=False
        )
        
        if response.status_code == 200:
            return {item.index: item.relevance_score for item in response.output.results}
        else:
            raise RuntimeError(f"Rerank 调用失败: {response.message}")

In [5]:
# 测试用例
test_query = "床舒服吗？"
test_documents = [
    "床很大很舒服，睡眠质量很好",
    "早餐种类丰富，味道不错",
    "床垫有点硬，睡得不太好",
    "交通很方便，打车很快"
]

reranker = Reranker(api_key=required_env["DASHSCOPE_API_KEY"])
reranker.rerank(test_query, test_documents)

{0: 0.8263690689666771,
 2: 0.6466967139441882,
 1: 0.2770358790522297,
 3: 0.23080660059017882}

### 4.2 多因子排序

In [6]:
class MultiFactorRanker:
    """多因子排序器：融合相关性、内容质量、时效性进行综合排序"""
    
    def __init__(self, reranker,
        
        # 排序权重
        w_relevance: float = 0.40,   # 相关性权重
        w_quality: float = 0.25,     # 评论质量分权重
        w_length: float = 0.05,      # 评论长度权重
        w_review: float = 0.05,      # 回复数权重
        w_useful: float = 0.05,      # 点赞数权重
        w_recency: float = 0.20,     # 时效性权重
        
        # 时效性参数
        base_decay: float = 0.5,     # 基础衰减率
        implied_boost: float = 0.5,  # implied 级别额外衰减增量
        clear_boost: float = 0.5,    #  clear  级别额外衰减增量（在 implied 基础上）
        half_life_days: int = 180    # 半衰期（天数）
        
    ):
        self.reranker = reranker
        
        # 排序权重
        self.w_relevance = w_relevance
        self.w_quality = w_quality
        self.w_length = w_length
        self.w_review = w_review
        self.w_useful = w_useful
        self.w_recency = w_recency
        
        # 时效性参数
        self.base_decay = base_decay
        self.implied_boost = implied_boost
        self.clear_boost = clear_boost
        self.half_life_days = half_life_days
    
    def rank(self, query: str, candidates: list[dict], time_sensitivity: str = None, topk: int = 10,
             today: datetime | None = None) -> tuple[list[dict], dict]:
        """
        多因子排序
        
        参数:
            query: 用户查询
            candidates: 混合检索返回的候选评论列表
            time_sensitivity: 时效性需求
            topk: 返回前 K 个结果
            today: 当前时间
        
        返回:
            (ranked_results, timing_info)
        """
        ranking_start = time.time()
        
        if not candidates:
            return [], {'total': 0, 'rerank': 0, 'scoring': 0}
        
        # 1. Rerank 打分
        rerank_start = time.time()
        documents = [c['comment'] for c in candidates]
        relevance_map = self.reranker.rerank(query, documents)
        rerank_time = time.time() - rerank_start
        
        # 2. 提取各特征值
        scoring_start = time.time()

        # 相关性
        relevance_score = np.array([relevance_map.get(i, 0) for i in range(len(candidates))])  # 取值范围 (0, 1)

        # 内容质量
        # (1) 评论质量分
        quality_score = np.array([c['metadata']['quality_score'] for c in candidates])
        norm_quality = quality_score / 10.0        # 取值范围 (0, 1)

        # (2) 评论长度
        comment_len = np.array([len(c['comment']) for c in candidates])
        log_comment_len = np.log(comment_len + 1)
        norm_length = log_comment_len / 7.51       # 取值范围 (0, 1)

        # (3) 回复数
        review_count = np.array([c['metadata']['review_count'] for c in candidates])
        log_review_count = np.log(review_count + 1)
        norm_review = log_review_count / 6.32      # 取值范围 (0, 1)
        
        # (4) 点赞数
        useful_count = np.array([c['metadata']['useful_count'] for c in candidates])
        log_useful_count = np.log(useful_count + 1)
        norm_useful = log_useful_count / 3.64      # 取值范围 (0, 1)
        
        # 时效性
        decay = self.base_decay
        if time_sensitivity == "implied":
            decay += self.implied_boost
        elif time_sensitivity == "clear":
            decay += self.implied_boost + self.clear_boost
        
        publish_date = pd.to_datetime([c['metadata']['publish_date'] for c in candidates])
        if not today:
            today = datetime.today()
        days_ago = (today - publish_date).days.values
        days_ago = np.maximum(days_ago, 0)
        recency_score = np.exp(-decay * days_ago / self.half_life_days)  # 取值范围 (0, 1)
        
        # 3. 计算综合得分并排序
        final_score = (
            self.w_relevance * relevance_score +
            self.w_quality * norm_quality +
            self.w_length * norm_length +
            self.w_review * norm_review +
            self.w_useful * norm_useful +
            self.w_recency * recency_score
        )
        sorted_index = np.argsort(final_score)[::-1]
        
        # 4. 构建结果
        ranked_results = []
        
        # 预计算 rerank_rank
        rerank_sorted_index = np.argsort(relevance_score)[::-1]
        rerank_rank = np.empty_like(rerank_sorted_index)
        rerank_rank[rerank_sorted_index] = np.arange(1, len(relevance_score) + 1)
        
        # 添加重排模块信息
        for rank, idx in enumerate(sorted_index[:topk], 1):
            c = candidates[idx]
            result = {
                **c,
                'rerank_score': float(relevance_score[idx]),
                'rerank_rank': int(rerank_rank[idx]),
                'final_score': float(final_score[idx]),
                'final_rank': rank,
                'feature_scores': {
                    'relevance': float(relevance_score[idx]),
                    'quality': float(norm_quality[idx]),
                    'log_comment_len': float(norm_length[idx]),
                    'log_review_count': float(norm_review[idx]),
                    'log_useful_count': float(norm_useful[idx]),
                    'recency': float(recency_score[idx])
                }
            }
            ranked_results.append(result)
        
        scoring_time = time.time() - scoring_start
        
        timing_info = {
            'total': time.time() - ranking_start,
            'rerank': rerank_time,
            'scoring': scoring_time
        }
        
        return ranked_results, timing_info

## 模块五：回复生成

In [7]:
class ResponseGenerator:
    """回复生成器：基于检索上下文生成最终回复"""
    
    def __init__(self, api_key: str, model: str = "deepseek-v3.2"):
        self.api_key = api_key
        self.model = model
    
    def generate(self, user_query: str, rewritten_queries: list[dict] = None, ranked_comments: list[dict] = None, summaries: list[dict] = None,
                 need_retrieval: bool = True, print_response: bool = True, today: datetime | None = None) -> tuple[str, float, float]:
        """
        生成回复
        
        参数:
            user_query: 用户原始查询
            rewritten_queries: 改写后的查询列表（含权重）
            ranked_comments: 排序后的 Top-K 评论
            summaries: 召回的摘要信息
            need_retrieval: 是否需要检索（意图识别结果）
            print_response: 是否需要打印流式输出
            today: 当前时间
        
        返回:
            (response_text, ttft_model, subsequent_time, generation_time)
        """
        start_time = time.time()
        
        # 直接回答
        if not need_retrieval:
            prompt = f"""
你是广州花园酒店的智能客服助手。

用户问题：{user_query}

请直接回答用户的问题。注意：
- 如果是问候或闲聊，友好回应
- 如果是通用问题，给出简洁准确的回答
- 语气要亲切专业
- 使用Markdown格式输出，不得出现 "```markdown", "```" 标记
"""

        # 基于检索回答
        else:
            if not today:
                today = datetime.today()
            date = f"{today.year}年{today.month}月{today.day}日"
            
            # 构建改写 Query 上下文
            queries_context = ""
            if rewritten_queries:
                queries_context += "【问题解析】\n系统识别到用户可能关注以下方面：\n"
                queries_context += "\n".join([f"- {q['query']}（意图权重为{q['weight']}）" for q in rewritten_queries])
                queries_context += "\n注意：权重信息是用来帮助你区分意图主次的，**不得**向用户输出权重相关信息。"
            
            # 构建评论上下文
            if ranked_comments:
                comments_context = "【相关用户评论】\n"
                for i, c in enumerate(ranked_comments, 1):
                    comments_context += f"""
【评论{i}】
评分: {c['metadata']['score']}（满分5分）
发布日期: {c['metadata']['publish_date']}
评论文本: {c['comment']}
点赞数: {c['metadata']['useful_count']}
回复数: {c['metadata']['review_count']}
房型: {c['metadata']['room_type']}
"""
            else:
                comments_context = "【未检索到相关用户评论】\n"
            
            # 构建摘要上下文
            summaries_context = ""
            if summaries:
                summaries_context += "【相关评论摘要】\n"
                for s in summaries:
                    summaries_context += f"""
【{s['metadata']['category']}类别摘要】
关键词: {s['metadata']['keywords']}
摘要: {s['summary']}
"""
                summaries_context += """
注意：评论摘要是用来给到你更丰富的概览信息的，但用户只能看到【相关用户评论】的引用而看不到摘要的引用，因此在回复中你可以给出摘要中的模糊信息，\
但**不得**过于精确因为用户无法溯源，也**不得**告诉用户你引用了摘要。若摘要中的信息与用户问题无关，直接忽略即可，**不需要**做出任何额外说明。
"""
            
            prompt = f"""
你是广州花园酒店的智能客服助手，需要基于用户评论为用户提供准确、高质量、有帮助、简洁的回答。

今天是：{date}

用户问题：{user_query}

{queries_context}

{comments_context}

{summaries_context}

【回答要求】
1. 综合以上评论信息，给出客观、全面的回答
2. 回答要有条理，突出重点
3. 如有正面和负面评价，都要提及，保持客观。注意给出的参考评论并不代表所有，切忌以偏概全给出“绝对化”的表述
4. 语气要专业、亲切
5. 回答长度适中，不要过于冗长
6. 不得大段或连续照抄用户评论，严禁全文都在引用用户评论却并没有思考提炼总结。相似内容能合并就合并，不要分开引用（合并后注意不得同时列出超过3条参考评论，使用“等”替代）
7. 一般来说越靠前的评论，其重要性越高，但你也可以自行判断自行选择
8. 不得在回复中罗列用户评论的具体日期，但当用户问题时效性敏感时，可以大致提一下参考评论的时间范围；当用户未表现出明显时效性需求时不要强行给出具体时间
9. 引用某一条评论独特内容时应指出评论几，供用户参考，但针对参考评论总体（如“多数住客……”等内容）或摘要进行归纳总结时无需指出参考了哪些评论
10. 不得同时列出超过3条参考评论，即禁止诸如“（评论1/3/5/7）”的表述。如需同时引用超过3条评论，则应输出“（评论1/3等）”，而不是将其全部列出。优先给出排名靠前的评论引用
11. 如果评论信息不足以回答问题，诚实说明
12. 使用Markdown格式输出，不得出现 "```markdown", "```" 标记

用户问题：{user_query}

请给出你的回答：
"""
        
        # 调用 LLM 生成回复（流式输出）
        completion = Generation.call(
            api_key=self.api_key,
            model=self.model,
            prompt=prompt,
            temperature=0.7,
            result_format="message",
            stream=True,
            incremental_output=True
        )
        
        response_content = ""
        ttft_model = 0
        subsequent_time = 0
        generation_time = 0
        
        for chunk in completion:
            if chunk.status_code != 200:
                raise RuntimeError(f"回复生成失败: {chunk.message}")
            
            message = chunk.output.choices[0].message
            if message.content:
                if not ttft_model:
                    ttft_model = time.time() - start_time        # 模型回复首字延迟
                    first_token_time = time.time()
                if print_response:
                    print(message.content, end="", flush=True)   # 直接打印输出
                response_content += message.content
        
        if print_response and response_content:
            print()

        if ttft_model:
            subsequent_time = time.time() - first_token_time     # 模型后续回复延迟
        
        generation_time = time.time() - start_time               # 模型回复总延迟
        
        return response_content, ttft_model, subsequent_time, generation_time

## 完整 RAG 系统

In [8]:
class HotelReviewRAG:
    """酒店评论 RAG 系统：完整的检索增强生成工作流"""
    
    def __init__(self, api_key: str, dashvector_api_key: str, dashvector_endpoint: str, data_dir: Path,
                 detection_model: str = "qwen-plus", expansion_hyde_model: str = "qwen-flash", generation_model: str = "deepseek-v3.2"):
        
        # 连接向量数据库
        dashvector_client = dashvector.Client(api_key=dashvector_api_key, endpoint=dashvector_endpoint)
        self.comments_collection = dashvector_client.get("comment_database")
        self.reverse_queries_collection = dashvector_client.get("reverse_query_database")
        
        chroma_db_path = data_dir / "chroma_db"
        chroma_client = chromadb.PersistentClient(path=str(chroma_db_path))
        self.summaries_collection = chroma_client.get_collection("summary_database")
        
        # 加载离线数据
        processed_dir = data_dir / "processed"
        self.inverted_index = InvertedIndex()
        self.inverted_index.load(processed_dir / "inverted_index.pkl")
        self.df_comments = pd.read_csv(processed_dir / "enriched_comments.csv", index_col=0)
        
        # 初始化各组件
        self.detection_client = LLMClient(api_key=api_key, model=detection_model, json=True)
        self.expansion_hyde_client = LLMClient(api_key=api_key, model=expansion_hyde_model, json=True)
        self.embedding_client = EmbeddingClient(api_key=api_key)
        self.intent_recognizer = IntentRecognizer(api_key=api_key)
        self.intent_detector = IntentDetector(self.detection_client, EXACT_ROOM_TYPES, FUZZY_ROOM_TYPES)
        self.intent_expander = IntentExpander(self.expansion_hyde_client)
        self.hyde_generator = HyDEGenerator(self.expansion_hyde_client)
        self.retriever = HybridRetriever(
            self.inverted_index, self.comments_collection, self.reverse_queries_collection,
            self.summaries_collection, self.embedding_client, self.df_comments, self.hyde_generator
        )
        self.reranker = Reranker(api_key=api_key)
        self.generator = ResponseGenerator(api_key=api_key, model=generation_model)
        
    def query(self, user_query: str,
              
        # 检索参数
        route_topk: int = 150,
        retrieval_topk: int = 100,
        ranking_topk: int = 10,

        # 是否启用意图扩展器
        enable_expansion: bool = True,
              
        # 五路召回开关
        enable_bm25: bool = True,
        enable_vector: bool = True,
        enable_reverse: bool = True,
        enable_hyde: bool = True,
        enable_summary: bool = True,
              
        # 是否启用重排模块
        enable_ranking: bool = True,

        # 是否需要生成最终回复并打印流式输出
        enable_generation: bool = True,
        print_response: bool = True,
              
        # 排序权重
        w_relevance: float = 0.40,
        w_quality: float = 0.25,
        w_length: float = 0.05,
        w_review: float = 0.05,
        w_useful: float = 0.05,
        w_recency: float = 0.20,
              
        # 时效性参数
        base_decay: float = 0.5,
        implied_boost: float = 0.5,
        clear_boost: float = 0.5,
        half_life_days: int = 180,
        today: datetime | None = TODAY
              
    ) -> dict:
        """
        处理用户查询并生成回复
        
        参数:
            user_query: 用户输入的查询
            route_topk: 每路召回数量
            retrieval_topk: 混合检索输出数量
            ranking_topk: 重排后输出数量
            enable_expansion: 是否启用意图扩展器
            enable_bm25 等: 是否启用该路召回
            enable_ranking: 是否启用重排模块
            enable_generation: 是否需要生成最终回复
            print_response: 是否需要打印流式输出
            w_relevance 等: 排序权重
            base_decay 等: 时效性参数
            today: 当前时间
        
        返回:
            {
                'response': 模型回复文本,
                'references': {
                    'comments': 参考的评论列表,
                    'summaries': 参考的摘要列表,
                    'hyde_responses': HyDE 生成的假设回复
                },
                'query_processing': {
                    'intent_recognition': 意图识别结果,
                    'intent_detection': 意图检测结果,
                    'intent_expansion': 意图扩展结果
                },
                'timing': {
                    'intent_recognition': 意图识别延迟,
                    'intent_detection': 意图检测延迟,
                    'intent_expansion': 意图扩展延迟,
                    'query_processing_total': 查询处理总延迟（不含 HyDE）,
                    'retrieval': 混合检索延迟详情,
                    'ranking': 重排延迟,
                    'ttft': 首字延迟,
                    'ttft_model': 模型回复首字延迟,
                    'subsequent': 模型后续回复延迟,
                    'generation': 模型回复总延迟,
                    'total': 总延迟
                }
            }
        """
        total_start = time.time()
        timing = {}
        if not today:
            today = datetime.today()
        
        # 一、查询处理
        query_processing_start = time.time()
        
        # 1. 意图识别
        intent_recognition_start = time.time()
        need_retrieval = self.intent_recognizer.recognize(user_query)
        timing['intent_recognition'] = time.time() - intent_recognition_start
        
        # 2. 意图检测与意图扩展（并行执行）
        intent_detection_result = None
        intent_expansion_result = None
        timing['intent_detection'] = 0
        timing['intent_expansion'] = 0
        
        if need_retrieval:
            if enable_expansion:
                with ThreadPoolExecutor(max_workers=2) as executor:
                    future_detect = executor.submit(self._timed_call, self.intent_detector.detect, user_query)
                    future_expand = executor.submit(self._timed_call, self.intent_expander.expand, user_query)
                    
                    intent_detection_result, timing['intent_detection'] = future_detect.result()
                    intent_expansion_result, timing['intent_expansion'] = future_expand.result()

            # 不启用意图扩展
            else:
                intent_detection_result, timing['intent_detection'] = self._timed_call(self.intent_detector.detect, user_query)
                intent_expansion_result, timing['intent_expansion'] = None, 0  # 后续将直接使用原始 Query
        
        timing['query_processing_total'] = time.time() - query_processing_start
        
        # 直接回答
        if not need_retrieval:
            if enable_generation:
                first_token_base = time.time() - total_start

                # 获取模型回复
                response, ttft_model, subsequent, generation = self.generator.generate(
                    user_query, need_retrieval=False, print_response=print_response, today=today
                )

                timing['ttft'] = first_token_base + ttft_model
                timing['ttft_model'] = ttft_model
                timing['subsequent'] = subsequent
                timing['generation'] = generation
                
            else:
                response = ""
                timing['ttft'] = 0
                timing['ttft_model'] = 0
                timing['subsequent'] = 0
                timing['generation'] = 0

            timing['total'] = time.time() - total_start
            
            return {
                'response': response,
                'references': {
                    'comments': [],
                    'summaries': [],
                    'hyde_responses': {}
                },
                'query_processing': {
                    'intent_recognition': need_retrieval,
                    'intent_detection': None,
                    'intent_expansion': None
                },
                'timing': timing
            }
        
        # 二、混合检索
        if enable_ranking:
            final_topk_for_retrieval = retrieval_topk  # 启用重排时，混合检索输出 retrieval_topk 条，排序后输出 ranking_topk 条
        else:
            final_topk_for_retrieval = ranking_topk    # 不启用重排时，混合检索直接输出 ranking_topk 条
        
        rewritten_queries = intent_expansion_result if intent_expansion_result else [{'query': user_query, 'weight': 1.0}]
        
        comments, summaries, retrieval_timing, hyde_results = self.retriever.retrieve(
            rewritten_queries,
            room_type=intent_detection_result.get('room_type'),
            fuzzy_room_type=intent_detection_result.get('fuzzy_room_type'),
            topk=route_topk,
            final_topk=final_topk_for_retrieval,
            enable_bm25=enable_bm25,
            enable_vector=enable_vector,
            enable_reverse=enable_reverse,
            enable_hyde=enable_hyde,
            enable_summary=enable_summary
        )
        timing['retrieval'] = retrieval_timing
        
        # 三、重排
        if enable_ranking:
            ranker = MultiFactorRanker(
                self.reranker,
                w_relevance=w_relevance,
                w_quality=w_quality,
                w_length=w_length,
                w_review=w_review,
                w_useful=w_useful,
                w_recency=w_recency,
                base_decay=base_decay,
                implied_boost=implied_boost,
                clear_boost=clear_boost,
                half_life_days=half_life_days
            )
            ranked_comments, ranking_timing = ranker.rank(
                user_query,
                comments,
                time_sensitivity=intent_detection_result.get('time_sensitivity'),
                topk=ranking_topk,
                today=today
            )
            timing['ranking'] = ranking_timing
        
        # 不启用重排
        else:
            ranked_comments = comments  # 直接使用混合检索结果
            timing['ranking'] = {'total': 0, 'rerank': 0, 'scoring': 0}
        
        # 四、回复生成
        if enable_generation:
            first_token_base = time.time() - total_start

            # 获取模型回复
            response, ttft_model, subsequent, generation = self.generator.generate(
                user_query,
                rewritten_queries=intent_expansion_result,
                ranked_comments=ranked_comments,
                summaries=summaries,
                need_retrieval=True,
                print_response=print_response,
                today=today
            )
            
            timing['ttft'] = first_token_base + ttft_model
            timing['ttft_model'] = ttft_model
            timing['subsequent'] = subsequent
            timing['generation'] = generation
            
        else:
            response = ""
            timing['ttft'] = 0
            timing['ttft_model'] = 0
            timing['subsequent'] = 0
            timing['generation'] = 0
        
        timing['total'] = time.time() - total_start
        
        # 五、构建返回结果
        processed_comments = []
        for c in ranked_comments:
            comment_data = {
                'comment_id': c['comment_id'],
                'comment': c['comment'],
                'rrf_score': c['rrf_score'],
                'rrf_rank': c['rrf_rank'],
                'route_ranks': c['route_ranks'],
                'metadata': c['metadata']
            }
            
            # 如果启用了重排，添加重排相关信息
            if enable_ranking:
                comment_data['rerank_score'] = c['rerank_score']
                comment_data['rerank_rank'] = c['rerank_rank']
                comment_data['final_score'] = c['final_score']
                comment_data['final_rank'] = c['final_rank']
                comment_data['feature_scores'] = c['feature_scores']
            
            processed_comments.append(comment_data)
        
        return {
            'response': response,
            'references': {
                'comments': processed_comments,
                'summaries': summaries,
                'hyde_responses': hyde_results
            },
            'query_processing': {
                'intent_recognition': need_retrieval,
                'intent_detection': intent_detection_result,
                'intent_expansion': intent_expansion_result
            },
            'timing': timing
        }
    
    def _timed_call(self, func, *args) -> tuple:
        """带计时的函数调用"""
        start = time.time()
        result = func(*args)
        return result, time.time() - start

In [9]:
# 初始化 RAG 系统
rag_system = HotelReviewRAG(
    api_key=required_env["DASHSCOPE_API_KEY"],
    dashvector_api_key=required_env["DASHVECTOR_API_KEY"],
    dashvector_endpoint=required_env["DASHVECTOR_HOTEL_ENDPOINT"],
    data_dir=DATA_DIR,
    detection_model=DETECTION_MODEL,
    expansion_hyde_model=EXPANSION_HYDE_MODEL,
    generation_model=GENERATION_MODEL
)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\22418\AppData\Local\Temp\jieba.cache
Loading model cost 0.588 seconds.
Prefix dict has been built successfully.


倒排索引已加载


- 关注 #6 召回路由（意图扩展作用）
- 关注 #4 召回路由（HyDE、反向召回作用）

In [10]:
# 测试用例 1（完整 RAG 系统）
test_query = "酒店的床舒服吗？睡眠质量怎么样？"
result = rag_system.query(test_query)
print_rag_result(result)

根据近期住客的反馈，广州花园酒店的床品舒适度和睡眠体验因人而异，并受到房间位置和外部环境的显著影响。

**关于床的舒适度：**
*   **正面评价居多：** 多数住客对床垫和床品给予了高度评价，常用“舒服柔软”、“一夜好眠”来形容。部分房型（如行政大床房）配备了乳胶床垫，获得了好评（评论6）。
*   **存在个体差异：** 床垫的软硬度感受存在主观差异。有住客认为床品非常舒服（评论1/3/4等），但也有住客（尤其是对腰部支撑有要求或习惯睡硬床的客人）反馈床垫“太软”，影响了睡眠（评论2）。酒店服务团队能够响应客人的需求，例如为觉得床垫过软的客人提供调整帮助（评论10）。

**关于睡眠质量：**
睡眠质量与床的舒适度相关，但更关键的影响因素是**房间的安静程度**。
*   **安静的房间体验良好：** 分配在面向内部花园、庭院或小区一侧的房间，住客普遍反馈“晚上睡觉特别安静”（评论4），能获得很好的休息。
*   **临街或近施工区域的房间噪音问题突出：** **这是近期影响睡眠质量最普遍的因素。** 许多住客指出，面向主干道或临近外部施工工地（如地铁施工）的房间，会受到车流、鸣笛声和施工噪音的干扰，严重影响睡眠，甚至导致“每天起床都疲惫不堪”（评论5）。酒店部分区域的隔音效果也存在差异，有客人反映能听到走廊或隔壁的声音。
*   **酒店的服务响应：** 针对噪音问题，部分客人反馈在提出诉求后，酒店客服部能够积极协助更换房间（评论6）。建议对睡眠环境要求高的客人，在办理入住时主动提出希望安排安静朝向的房间。

**总结建议：**
花园酒店的床品整体上获得了较多住客的认可，但睡眠体验存在不确定性，主要取决于您被分配到的具体房间位置。为了获得更佳的睡眠体验，我们建议您在预订或办理入住时，提前说明您对安静环境的需求，以便我们尽力为您安排面向花园等安静区域的客房。

⏱️  延迟统计:
  • 查询处理（不含HyDE）: 1.350s
    • 意图识别: 0.400s
    • 意图检测: 0.948s
    • 意图扩展: 0.938s
  • 混合检索: 3.408s
    • 文本召回: 0.034s
    • 向量召回: 0.388s
    • 反向召回: 0.382s
    • HyDE召回: 3.080s（生成 2.668s + 检索 

In [11]:
# 测试用例 2（不启用意图扩展器）
result_no_expansion = rag_system.query(test_query, enable_expansion=False)
print_rag_result(result_no_expansion)

根据近期住客的反馈，广州花园酒店的床品舒适度和睡眠体验整体评价较好，但具体感受会受到**客房位置**和**个人偏好**的影响。

**关于床品舒适度：**
- **多数住客给予了高度评价**，认为床垫和床品“舒服柔软”、“非常舒服”、“真的是一夜好眠”（评论1/2/4/6/7等）。有住客甚至描述为“住过的最舒服的床”（评论3）。
- 酒店在服务上较为灵活，如果您对床垫软硬度有特殊要求，可以向酒店反映。例如，曾有住客反馈床垫偏软后，酒店服务人员及时为其进行了更换（评论10）。

**关于睡眠质量：**
睡眠质量不仅与床品有关，也与**房间安静程度**密切相关，近期住客的体验存在差异：
- **安静的房间能提供优质睡眠**：选择面向酒店内部花园或小区的客房，通常环境安静，有助于良好休息（评论3/4）。
- **临街房间可能存在噪音干扰**：部分朝向马路的房间，夜间可能会受到车流声影响。有睡眠较浅的住客反映因此休息不佳（评论5）。此外，极个别情况下，夜间施工等外部噪音也可能干扰睡眠（评论8）。
- **客房维护状况**：酒店部分房间因开业时间较长，设施略显陈旧（如评论1提及墙纸旧），但多数住客反馈房间干净整洁，不影响核心居住体验。

**总结与建议：**
酒店的床品普遍受到好评，是确保舒适睡眠的有利条件。为了获得更佳的睡眠体验，建议您在预订或办理入住时，可以根据自己对安静环境的需求，向酒店提出希望安排**不临街、面向花园或内部**的客房。如果对床垫软硬度有特别偏好，入住后也可及时与前台沟通。

⏱️  延迟统计:
  • 查询处理（不含HyDE）: 1.484s
    • 意图识别: 0.458s
    • 意图检测: 1.025s
    • 意图扩展: 0.000s
  • 混合检索: 4.502s
    • 文本召回: 0.014s
    • 向量召回: 0.330s
    • 反向召回: 0.326s
    • HyDE召回: 4.199s（生成 3.722s + 检索 0.472s）
    • 摘要召回: 0.285s
    • RRF融合: 0.020s
  • 重排: 0.575s（Rerank 0.573s + 排序 0.002s）
  • 模型回复: 10.602s
    • 首字延迟: 0.477s
    • 后续回复: 10.12

- 合适的首字延迟

In [12]:
# 测试用例 3（不启用 HyDE 召回）
result_no_hyde = rag_system.query(test_query, enable_hyde=False)
print_rag_result(result_no_hyde)

您好，关于广州花园酒店的床铺舒适度和整体睡眠质量，根据近期客人的反馈，情况如下，供您参考：

**床铺舒适度普遍获好评**
多数入住客人对床品给予了高度评价，认为床垫和床上用品柔软舒适，有助于获得良好的睡眠。例如，有客人提到“床品很舒服睡眠很好”（评论1），也有客人称赞“真的是住过的最舒服的床！！”（评论3）。部分房型（如行政大床房）的床垫为乳胶材质，并可根据需求选择枕头，提升了睡眠体验（评论6）。

**睡眠质量受房间位置影响显著**
需要注意的是，睡眠质量与您入住的**具体房间位置和朝向**密切相关，客人反馈呈现两极分化：
*   **安静的房间体验佳**：分配到面向内部花园、庭院或小区的房间，客人普遍反馈“晚上睡觉特别安静”（评论3），能享受到静谧的休息环境。
*   **临街房间可能影响睡眠**：如果房间朝向主干道（如环市路），部分客人反映夜间车流噪音可能比较明显，对睡眠浅的客人有一定影响（评论4）。此外，酒店周边可能存在市政施工，这也是影响部分客房安静程度的一个外部因素（评论6）。

**给您的建议**
为了确保您能获得优质的睡眠，我们建议：
1.  **办理入住时主动沟通**：如果您对睡眠环境要求较高，可以在前台办理入住时，礼貌地提出希望安排一个安静朝向（如面向花园或小区）的房间。
2.  **参考房型选择**：从评论看，**花园大床房**、**红棉大床套房**等房型的客人都对床的舒适度有正面评价。部分套房或经过翻新的房间在设施和隔音上可能更有优势（评论3/6）。

总结来说，花园酒店的床铺本身舒适度受到客人认可，但最终的睡眠质量很大程度上取决于房间的具体位置。通过主动与酒店沟通您的需求，有助于获得更理想的入住体验。

⏱️  延迟统计:
  • 查询处理（不含HyDE）: 1.443s
    • 意图识别: 0.422s
    • 意图检测: 0.920s
    • 意图扩展: 1.018s
  • 混合检索: 0.437s
    • 文本召回: 0.010s
    • 向量召回: 0.388s
    • 反向召回: 0.397s
    • HyDE召回: 0.000s
    • 摘要召回: 0.315s
    • RRF融合: 0.026s
  • 重排: 0.578s（Rerank 0.575s + 排序 0.003s）
  • 

- 关注评论长度（重排作用）

In [13]:
# 测试用例 4（不启用重排模块）
result_no_ranking = rag_system.query(test_query, enable_ranking=False)
print_rag_result(result_no_ranking)

关于广州花园酒店的床铺舒适度与睡眠质量，综合近期住客的反馈，为您提供以下信息：

**床铺舒适度方面：**
多数住客对床品给予了积极评价，常用“舒适”、“松软温暖”、“保障优质睡眠”来形容（评论1/2/8等）。不过，床垫的软硬度体验因人而异。部分住客，尤其是有腰椎问题的客人，反馈床垫**偏软**（评论5），而习惯睡硬床的客人也可能感到不适（评论6）。这表明酒店的床垫风格可能更偏向于柔软支撑型。

**影响睡眠质量的关键因素：**
睡眠体验不仅取决于床，**房间的安静程度**是另一项关键影响因素，且评价呈现明显的两极分化：
*   **正面体验**：许多住客表示房间隔音良好，即使临街也能“睡得很好”（评论3），或直接称赞“房间也很安静”（评论2）。
*   **潜在风险**：**外部噪音**是影响睡眠的最主要负面因素。近期有住客反映，若房间朝向主干道或临近外部施工区域，可能会受到**车流噪音**或**夜间施工噪音**的严重干扰（评论4/10）。此外，酒店内部如走廊声响、邻近房间活动等**隔音问题**也偶有提及。

**给您的建议：**
1.  **床铺偏好**：如果您或同行家人偏好硬床或对腰椎支撑有较高要求，建议在预订或入住时提前咨询前台，看是否能提供硬质床垫或辅助支撑（如评论5中客人的建议）。
2.  **确保安静**：如果您对噪音比较敏感，**强烈建议在办理入住时主动提出需要安静房间的诉求**，例如要求面向内部花园或庭院的房间，并再次确认房间不临施工区域或主干道。
3.  **积极沟通**：若入住后发现噪音问题影响休息，及时联系前台沟通换房，酒店通常愿意为此提供协助（评论7）。

总结来说，花园酒店的床铺普遍获得好评，但睡眠整体体验在很大程度上取决于您所分配到的具体房间位置及其安静程度。通过事前沟通明确需求，能有效提升获得舒适睡眠体验的概率。

⏱️  延迟统计:
  • 查询处理（不含HyDE）: 1.429s
    • 意图识别: 0.460s
    • 意图检测: 0.850s
    • 意图扩展: 0.966s
  • 混合检索: 5.880s
    • 文本召回: 0.027s
    • 向量召回: 0.389s
    • 反向召回: 0.373s
    • HyDE召回: 5.558s（生成 5.134s + 检索 0.430s）
   

- 用于后续超参数贝叶斯优化

In [14]:
# 测试用例 5（仅检索不生成最终回复）
result_no_gen = rag_system.query(test_query, enable_generation=False)
print_rag_result(result_no_gen)


⏱️  延迟统计:
  • 查询处理（不含HyDE）: 1.462s
    • 意图识别: 0.420s
    • 意图检测: 0.889s
    • 意图扩展: 1.037s
  • 混合检索: 3.220s
    • 文本召回: 0.017s
    • 向量召回: 0.375s
    • 反向召回: 0.375s
    • HyDE召回: 2.901s（生成 2.362s + 检索 0.516s）
    • 摘要召回: 0.293s
    • RRF融合: 0.028s
  • 重排: 0.525s（Rerank 0.523s + 排序 0.002s）
  • 模型回复: 0.000s
    • 首字延迟: 0.000s
    • 后续回复: 0.000s
  • 首字延迟: 0.000s
  • 总延迟: 5.207s

🔍 查询处理:
  • 意图检测: {'room_type': None, 'fuzzy_room_type': None, 'time_sensitivity': None}
  • 意图扩展:
      - 酒店的床垫舒适度如何？是否有助于提升睡眠质量？ (weight=0.6)
      - 酒店房间的隔音效果怎么样？是否会影响睡眠？ (weight=0.4)

🔮 HyDE 假设回复:
  • Q0-H0: 床垫真的太惊艳了！我一向对睡眠环境很敏感，但这次在这家酒店的夜睡得特别沉，醒来后脖子也不酸。床面软硬适中，贴合身体曲线，尤其适合像我这样久坐办公的人。清晨阳光照进来时，我还赖在床上不想起，简直是旅行中最值得的投入。
  • Q0-H1: 作为经常出差的商务人士，我对床品要求很高。这家酒店的床垫不仅支撑力好，还带有一层记忆棉，翻身时几乎没有干扰。我一整晚都没被硌醒，第二天精神状态比平时好太多。强烈推荐给需要高质量休息的旅客，绝对是加分项。
  • Q0-H2: 说实话，床垫让我有点失望。躺上去感觉太硬，腰背压得不舒服，半夜翻来覆去，最后靠抱枕头才勉强入睡。虽然房间其他方面都很不错，但睡眠体验明显受影响。希望下次能升级一下床具，否则会影响整体入住感受。
  • Q1-H0: 房间隔音效果出乎意料得好！晚上11点后走廊里还有人走动，但完全听不到声音，连隔壁的电视声都像隔着一层墙。我睡得特别沉，第二天精神满满，

In [15]:
# 测试用例 6（直接回答）
test_query_direct = "你好，今天天气怎么样？"
result_direct = rag_system.query(test_query_direct)
print_rag_result(result_direct)

您好！我主要专注于广州花园酒店相关的服务咨询，比如客房预订、餐饮信息、设施使用等问题。关于天气等实时信息，建议您通过天气预报应用或网站查询最新情况，这样会更准确哦。

如果您需要酒店方面的帮助，请随时告诉我，我会尽力为您解答！😊

⏱️  延迟统计:
  • 查询处理（不含HyDE）: 0.354s
    • 意图识别: 0.354s
    • 意图检测: 0.000s
    • 意图扩展: 0.000s
  • 模型回复: 2.673s
    • 首字延迟: 0.399s
    • 后续回复: 2.274s
  • 首字延迟: 0.754s
  • 总延迟: 3.029s

🔍 未触发检索，直接回答

🔮 未启用 HyDE 召回

📚 未启用摘要召回

🏆 未召回评论


In [16]:
# 测试用例 7（房型约束）
test_query_room = "花园大床房的窗户朝向如何？"
result_room = rag_system.query(test_query_room)
print_rag_result(result_room)

关于花园大床房的窗户朝向，综合多位住客的反馈，其景观与朝向主要分为两种类型，您可以根据自己的偏好进行选择或向酒店咨询。

**1. 面向酒店内部花园（瀑布景观）**
这是酒店的特色景观之一。部分花园大床房的窗户朝向酒店内部，可以欣赏到后花园的瀑布、小桥流水及岭南园林景致（评论2/3/5等）。这种朝向的房间通常环境清幽，更具度假感。但请注意，有住客提到瀑布的水声在特定房间可能较为明显，如果对安静度要求极高，建议选择高楼层或提前说明需求（评论3/6）。

**2. 面向外部街道（城市景观）**
另一部分房间的窗户朝向外部街道，例如环市东路一侧。这种朝向可能带来更开阔的城市视野。但需注意，由于酒店周边曾有地铁施工（评论1），以及日常车流，临街房间可能会受到一定的交通噪音影响。追求安静的住客在办理入住时，可以主动询问并尝试选择非临街或高楼层房间（评论1/3）。

**温馨提示：**
*   房间的具体朝向和景观可能因楼层、位置和酒店当期房态而有所不同。
*   如果您对景观或安静度有特别偏好，最有效的方式是在**办理入住时直接向酒店前台提出**，工作人员通常会尽力根据您的需求和当日房况进行安排（评论1/3/7等）。
*   酒店的部分客房近期经过翻新，设施与环境得到了提升（评论1/9），您也可以就此进行咨询。

希望以上信息能帮助您更好地了解房间情况。

⏱️  延迟统计:
  • 查询处理（不含HyDE）: 1.349s
    • 意图识别: 0.412s
    • 意图检测: 0.936s
    • 意图扩展: 0.794s
  • 混合检索: 2.990s
    • 文本召回: 0.003s
    • 向量召回: 0.338s
    • 反向召回: 0.335s
    • HyDE召回: 2.704s（生成 2.279s + 检索 0.418s）
    • 摘要召回: 0.273s
    • RRF融合: 0.012s
  • 重排: 0.520s（Rerank 0.519s + 排序 0.001s）
  • 模型回复: 9.803s
    • 首字延迟: 0.926s
    • 后续回复: 8.877s
  • 首字延迟: 5.786s
  • 总延迟: 14.663s

🔍 查询处理:
  • 意图检测: {'room_type': '花园大

In [17]:
# 测试用例 8（时效性敏感）
test_query_time = "现在早餐怎么样？"
result_time = rag_system.query(test_query_time)
print_rag_result(result_time)

您好！关于广州花园酒店的早餐，根据近期（2025年1月至4月）住客的评论，我们为您整理了以下信息，供您参考：

### 🍽️ 早餐概况
酒店早餐主要在两个地点供应，各具特色：
*   **顶楼旋转餐厅**：可边用餐边欣赏城市景色，环境备受好评（评论1/2/7等）。
*   **瀑布景观餐厅/行政酒廊外**：部分住客提到可以在花园或瀑布旁用餐，环境惬意、独特（评论6/8）。

### 👍 正面评价
多数近期住客对早餐体验表示满意，主要集中在：
*   **用餐环境**：旋转餐厅的城市景观和瀑布餐厅的花园景致是最大亮点，被认为体验感很好。
*   **食物品质与种类**：有住客评价早餐“品种很多，味道很棒”（评论2）、“是目前吃过最满意的”（评论5）。也有住客认为“品种虽然不多但是精致”（评论4）。
*   **服务体验**：有住客提到服务员会帮忙冲咖啡，服务到位（评论6/10）。

### 👎 需留意之处
同时，也有部分住客提出了以下几点，供您知悉：
*   **高峰期排队**：在入住客人较多时，早餐可能出现需要排队的情况（评论2）。
*   **口味与性价比差异**：个别住客认为菜品口味“很一般”，甚至表示“不如快捷酒店”，并提到了咖啡机故障影响体验的问题（评论3/9）。
*   **品种丰富度**：评价存在分化，有住客认为丰富，也有住客认为品种不算多但精致。

### 💎 总结建议
综合来看，广州花园酒店的早餐以其**独特优美的用餐环境**获得了住客的普遍认可。食物方面，近期评价呈现分化，多数反馈积极，但确实存在关于口味、性价比及高峰期排队的不同意见。
如果您对环境体验有较高要求，这里的早餐会是不错的选择。若对菜品丰富度或性价比特别关注，建议在入住时向工作人员咨询最新的早餐菜单与安排。

希望以上信息对您有帮助！祝您入住愉快。

⏱️  延迟统计:
  • 查询处理（不含HyDE）: 1.664s
    • 意图识别: 0.433s
    • 意图检测: 0.934s
    • 意图扩展: 1.225s
  • 混合检索: 4.085s
    • 文本召回: 0.021s
    • 向量召回: 0.793s
    • 反向召回: 0.792s
    • HyDE召回: 3.374s（生成 2.592s + 检索 0.776s）
    • 摘要召回:

## 总结

第四、五模块（重排与回复生成）及系统集成已全部完成

**关键组件**：
- Reranker: Qwen3-Rerank 相关性打分
- MultiFactorRanker: 多因子排序（相关性、内容质量、时效性）
- ResponseGenerator: （基于检索上下文）生成回复
- HotelReviewRAG: 集成模块一至五的完整工作流

**重排策略**：
- 模型选择：线性加权融合
- 评分公式：`Final_Score = Σ (Weight_i × Norm_Score_i)`
- 评分维度：
    - 相关性 (40%)：
        - 基于 Qwen3-Rerank 的 Query-Document 语义打分
    - 内容质量 (40%)：
        - 评论质量分 (25%)：基于预计算的 `quality_score` (归一化)
        - 其他指标 (15%)：评论长度 (5%)、回复数 (5%)、点赞数 (5%) 的对数归一化
    - 时效性 (20%)：
        - 基于发布日期的指数衰减 (`Exp(-decay × days / half_life)`)
        - 基础衰减：`base_decay = 0.5` (半衰期 180 天)
        - 意图增强：根据 `time_sensitivity` ("implied"/"clear") 动态增加衰减系数，使近期评论权重显著提升

**回复生成策略**：
- 上下文构建：
    - Query 解析：包含改写后的 Query 及其意图权重
    - 参考评论：Top-K 重排后的完整评论（含评分、房型、日期等）
    - 相关摘要：检索到的类别摘要
- 提示词工程：
    - 客观性：需同时涵盖正面与负面评价，避免以偏概全
    - 引用规范：合并相似观点，引用时需明确指出引用评论序号
    - 时效性响应：仅在用户意图敏感时强调时间信息
    - 摘要使用：利用摘要提供模糊概览，补充宏观视角
- 支持流式输出
    - 计算首字延迟 (TTFT)

**完整 RAG 系统**：
- 工作流：意图识别 -> 并行意图检测与扩展 -> 五路混合检索 -> 多因子重排 -> 上下文组装 -> LLM 生成
- 参数配置：
    - `retrieval_topk=100` (混合检索输出)
    - `ranking_topk=10` (重排输出)
- 全链路监控：
    - 记录各阶段延迟：查询处理 (意图识别/检测/扩展)、混合检索 (各路召回/HyDE生成/RRF融合)、重排 (Rerank/排序)、回复生成 (首字延迟/总延迟)
    - 保存完整中间状态 (References, Query Processing) 用于调试